[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giordamaug/EG-identification---Data-Science-in-App-Springer/blob/main/notebook/EssentialGenes_N2VMLP.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giordamaug/EG-identification---Data-Science-in-App-Springer/main?filepath=notebook%2FEssentialGenes_N2VMLP.ipynb)

# Loading required libraries

### Install Pytorch libraries

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if not IN_COLAB:
    !pip install -q pandas
    !pip install -q pandas
    !pip install -q sklearn
    !pip install -q imblearn
    !pip install -q xgboost
    !pip install -q tqdm
    !pip install -q torch
    
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

In [6]:
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import pandas as pd
import torch
def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

# Download dataset from Github

In [4]:
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/ppi.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/labels.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/bio_attributes.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/net_attributes.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/gtex_attributes.csv

--2022-05-04 10:15:22--  https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/ppi.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3966521 (3.8M) [text/plain]
Saving to: ‘ppi.csv’

ppi.csv             100%[===================>]   3.78M  --.-KB/s    in 0.07s   

2022-05-04 10:15:22 (55.4 MB/s) - ‘ppi.csv’ saved [3966521/3966521]

--2022-05-04 10:15:22--  https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent,

# Load the label
Only a subset of genes are selected for classification:
+ genes belonging to CS0 group, that are labeled as Essential (E);
+ genes belonging to CS6, CS7, ..., CS9 groups, that are labeled as Not-Essential (NE).

All remaining genes belong to intermediate groups (CS1-CS5) and are considered undetermined (label ND) 

In [7]:
labels = pd.read_csv("labels.csv", index_col='name')
labels = labels[labels["CS0_vs_CS6-9"].isin(['E', 'NE']) == True]       # drop any gene with undefined (ND) label
genes = labels.index.values                                             # get genes with defined labels (E or NE)
print(f'Selected {len(genes)} genes')

Selected 3814 genes


## Encode the labels
String labels E and Ne are respectively encoded to 0 and 1.
The array `y` containes numeric labels of genes.

In [8]:
from sklearn import preprocessing
from collections import Counter
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(labels['CS0_vs_CS6-9'].values)  
classes_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(classes_mapping, Counter(y))

{'E': 0, 'NE': 1} Counter({1: 3069, 0: 745})


# Load attributes to be used
We identified three sets of attributes:
1. bio attributes, related to gene information (such as, expression, etc.)
2. net attributes, derived from role of gene/node in the network (such as, degree, centrality, etc.)
3. GTEX-* attribute, additional biological information of genes 
Based on user selection, the node attributes are appended in a single matrix of attributes (`x`)

In the attribute matrix `x` there can be NaN or Infinite values. They are corrected as it follow:
+ NaN is replaced by the mean in the attribute range, 
+ Infinte value is replaced by the maximum in the range.

After Nan and Infinite values fixing, the attributes are normalized with Z-score or MinMax normalization functions.

At the end, only nodes (genes) with E or NE labels are selected for the classification

In [9]:
#@title Choose attributes { form-width: "20%" }
normalize_node = "zscore" #@param ["", "zscore", "minmax"]
bio = True #@param {type:"boolean"}
gtex = True #@param {type:"boolean"}
net = True #@param {type:"boolean"}
variable_name = "bio"
bio_df = pd.read_csv("bio_attributes.csv", index_col='name') if bio else pd.DataFrame()
gtex_df = pd.read_csv("gtex_attributes.csv", index_col='name') if gtex else pd.DataFrame()
net_df = pd.read_csv("net_attributes.csv", index_col='name') if net else pd.DataFrame()
x = pd.concat([bio_df, gtex_df, net_df], axis=1)
print(f'Found {x.isnull().sum().sum()} NaN values and {np.isinf(x).values.sum()} Infinite values')
for col in x.columns[x.isna().any()].tolist():
  mean_value=x[col].mean()          # Replace NaNs in column with the mean of values in the same column
  if mean_value is not np.nan:
    x[col].fillna(value=mean_value, inplace=True)
  else:                             # otherwise, if the mean is NaN, remove the column
    x = x.drop(col, 1)
if normalize_node == 'minmax':
  print("X attributes normalization (minmax)...")
  x = (x-x.min())/(x.max()-x.min())
elif normalize_node == 'zscore':
  print("X attributes normalization (zscore)...")
  x = (x-x.mean())/x.std()
x = x.loc[genes]
print(f'New attribute matrix x{x.shape}')

Found 15919 NaN values and 0 Infinite values
X attributes normalization (zscore)...
New attribute matrix x(3814, 119)


# Load the PPI+MET network
The PPI networks is loaded from a CSV file, where
*   `A` is the column name for edge source (gene name)
*   `B` is the column name for edge target (gene name)
*   `weight` is the column name for edge weight
Only some method use the PPI netoworks, as an example all GCN methods, and Node2Vec.

The PPI+MET network is reduced by removing genes with undetermined labels

In [12]:
ppi = pd.read_csv('ppi.csv')                                               # read PPI+MET network from CSV file
ppi = ppi.loc[((ppi['A'].isin(genes)) & (ppi['B'].isin(genes)))]           # reduce network only to selected nodes/genes
idxlbl = labels.reset_index(drop=True)
idxlbl['name'] = labels.index
map_gene_to_idx = { v['name']: i  for i,v in idxlbl.to_dict('Index').items() }
vfunc = np.vectorize(lambda t: map_gene_to_idx[t])
edges_index = torch.from_numpy(vfunc(ppi[['A','B']].to_numpy().T)) 

# k-fold cross validation with Node2Vec

### MLP model

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Loss():
    def __init__(self, y):
        self.y = y
        self.pos_mask = y == 1
        self.neg_mask = y == 0

    def __call__(self, out):
        pos_mask = self.pos_mask
        neg_mask = self.neg_mask
        loss_p = F.binary_cross_entropy_with_logits(
            out[pos_mask].squeeze(), self.y[self.pos_mask].to(DEVICE))
        loss_n = F.binary_cross_entropy_with_logits(
            out[neg_mask].squeeze(), self.y[neg_mask].to(DEVICE))
        loss = loss_p + loss_n
        return loss

def mlp_fit_predict(train_x, train_y, test_x, val=None, return_val_probs=False, log=False):
    epochs = 1000

    in_feats = train_x.shape[1]
    model = nn.Sequential(
        nn.Linear(in_feats, 32),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(32, 1))
    optimizer = torch.optim.Adam(model.parameters())

    lossf = Loss(train_y)

    if val is not None:
        val_x, val_y = val
        lossf_val = Loss(val_y)

    model.train()
    model.to(DEVICE)

    patience, cur_es = 3, 0
    val_loss_old = np.Inf

    for i in range(epochs):
        out = model(train_x)
        loss = lossf(out)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i % 10) == 0:
            if val is not None:
                model.eval()
                with torch.no_grad():
                    loss_val = lossf_val(model(val_x))
                if log: print(f'{i}. Train loss:', loss.detach().cpu().numpy(), ' |  Val Loss:', loss_val.detach().cpu().numpy())
                model.train()

                if val_loss_old < loss_val:
                    cur_es += 1
                else:
                    cur_es = 0
                val_loss_old = loss_val

                if cur_es == patience:
                    break

    model.eval()
    with torch.no_grad():
        out = model(test_x).cpu()
    probs = torch.sigmoid(out).numpy()

    if return_val_probs:
        with torch.no_grad():
            out = model(val_x).cpu()
        val_probs = torch.sigmoid(out).numpy()

        return probs, val_probs

    return probs

### Node2vec model

In [ ]:
from sklearn.svm import SVC
from torch_geometric.nn.models import Node2Vec
import torch.optim as optim
import torch_cluster
PARAMS = {
    'embedding_dim': 128,
    'walk_length': 64,
    'context_size': 64,
    'walks_per_node': 64,
    'num_negative_samples': 1,
}
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_epoch(n2v, n2v_loader, n2v_optimizer, X, train_y, train_mask, val_y, val_mask, test_mask, epochs=100, log=False):

    X = X.to(DEVICE)
    train_y = train_y.to(DEVICE)
    val_y = val_y.to(DEVICE)
    Z = None

    n2v.train()
    for i in range(epochs):
        n2v_train_loss = 0

        for (pos_rw, neg_rw) in n2v_loader:
            n2v_optimizer.zero_grad()
            loss = n2v.loss(pos_rw.to(DEVICE), neg_rw.to(DEVICE))
            loss.backward()
            n2v_optimizer.step()
            n2v_train_loss += loss.data.item()
        if log: print(f'Epoch {i}. N2V Train_Loss:', n2v_train_loss)
    n2v.eval()
    Z = n2v().detach()

    if X is None:
        train_x = Z[train_mask]
        val_x = Z[val_mask]
        test_x = Z[test_mask]
    elif Z is not None:
        train_x = torch.cat([Z[train_mask], X[train_mask]], dim=1)
        val_x = torch.cat([Z[val_mask], X[val_mask]], dim=1)
        test_x = torch.cat([Z[test_mask], X[test_mask]], dim=1)
    else:
        train_x = X[train_mask]
        val_x = X[val_mask]
        test_x = X[test_mask]
    if log: print('train_X.shape', train_x.shape)

    probs, val_probs = mlp_fit_predict(
        train_x, train_y, test_x, val=(val_x, val_y), return_val_probs=True)
    val_roc_auc = roc_auc_score(val_y.cpu().numpy(), val_probs)

    if log: print('Validation ROC_AUC:', val_roc_auc)
    return probs, val_roc_auc


def n2v_fit_predict(edge_index, X, train_y, train_mask, val_y, val_mask, test_mask, epochs=100, log=False):
    n2v = Node2Vec(edge_index, **PARAMS).to(DEVICE)
    n2v_loader = n2v.loader(batch_size=128, shuffle=True, num_workers=0)
    n2v_optimizer = optim.Adam(n2v.parameters(), lr=LR)

    probs, val_roc_auc = train_epoch(
        n2v, n2v_loader, n2v_optimizer, X, train_y, train_mask, val_y, val_mask, test_mask, epochs=epochs, log=log)

    return probs


### Validate

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from tqdm import tqdm
from sklearn.metrics import *

set_seed(1)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NFOLDS = 5
LR = 1e-2
WEIGHT_DECAY = 5e-4
EPOCHS = 50

X = torch.tensor(x.to_numpy(), dtype=torch.float)
kf = KFold(n_splits=NFOLDS)
cma = np.array([[0,0],[0,0]])
columns_names = ["Accuracy","BA", "Sensitivity", "Specificity","MCC", 'CM']
scores = pd.DataFrame(columns=columns_names)
mm = np.array([], dtype=np.int)
predictions = np.array([])
for fold, (train_index, test_idx) in enumerate(tqdm(kf.split(np.arange(len(X))), total=kf.get_n_splits(), desc=f"{NFOLDS}-fold")):
    train_idx, val_idx = train_test_split(train_index, test_size=0.05, stratify=y[train_index])
    mm = np.concatenate((mm, test_idx))
    train_y = torch.tensor(y[train_idx], dtype=torch.float)
    val_y = torch.tensor(y[val_idx], dtype=torch.float)
    test_y = torch.tensor(y[test_idx], dtype=torch.float).to(DEVICE)
    train_x = torch.tensor(X[train_idx], dtype=torch.float).to(DEVICE)
    val_x = torch.tensor(X[val_idx], dtype=torch.float).to(DEVICE)
    test_x = torch.tensor(X[test_idx], dtype=torch.float).to(DEVICE)
    probs = n2v_fit_predict(edges_index, X, train_y, train_idx, val_y, val_idx, test_idx, epochs=EPOCHS, log=False)
    preds = (probs > 0.5) * 1
    predictions = np.concatenate((predictions, preds.ravel()))
    cm = confusion_matrix(test_y.cpu().numpy(),preds)
    cma += cm
    scores = scores.append(pd.DataFrame([[accuracy_score(test_y.cpu().numpy(), preds), balanced_accuracy_score(test_y.cpu().numpy(), preds), 
        cm[0,0]/(cm[0,0]+cm[0,1]), cm[1,1]/(cm[1,0]+cm[1,1]), 
        matthews_corrcoef(test_y.cpu().numpy(), preds), cm]], columns=columns_names, index=[fold]))
df_scores = pd.DataFrame(scores.mean(axis=0)).T
df_scores.index=[f'N2V']
df_scores['CM'] = [cma]
df_scores

# Print predictions

In [ ]:
p = np.zeros(len(y))
p[mm] = predictions
labels['predictions'] = ['NE' if x>0 else 'E' for x in p]
labels